<a href="https://colab.research.google.com/github/kostique23/SpeechRecognitionAccuracy/blob/main/SpeechRecognitionAccuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек

!pip install -q SpeechRecognition pydub
!apt-get update && apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 61.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,642 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/graph

In [ ]:
#@title Импорт библиотек

import speech_recognition as sr
from pydub import AudioSegment
from pydub.generators import WhiteNoise
import os
import io
import numpy as np
import textwrap
from IPython.display import Audio

In [ ]:
#@title Загрузка данных

!wget https://storage.yandexcloud.net/homework/hw49.zip
!unzip hw49.zip -d hw49

REFERENCE = {

    "bu_ispugalsia.mp3": "Бу! Испугался? Не бойся, я друг, я тебя не обижу. Иди сюда, иди ко мне, сядь рядом со мной, посмотри мне в глаза. Ты видишь меня? Я тоже тебя вижу. Давай смотреть друг на друга до тех пор, пока наши глаза не устанут. Ты не хочешь? Почему? Что-то не так?",
    "pomogi_pj_s_domashkoi.mp3": "Помоги пж с дом... с домашкой, помоги пж с домашкой. Я обвела то, что надо. Пожалуйста помоги. Помоги, напиши ответ, помоги пожалуйста.",
    "nu_tipa_lol.mp3": "И что? И что? Ну и че типа с этого? Ну типа лол как бы эм... ну боже типа лол ну типа ну как бы типа ну и что типа я имею право типа ну делать то что типа я захочу как бы ну лол я считаю что типа если у тебя нормальная самооценка как бы эм ты понимаешь ну типа ну прикол просто в том что у меня аргументов нет? Боже чел я просто не собираюсь ну как-бы типа ну ну оправдываться типа ну.",
    "strashno_ochen_strashno.mp3": "Страшно, очень страшно. Мы не знаем, что это такое. Если бы мы знали, что это такое, мы не знаем, что это такое.",
    "silnaya_nezavisimaya.mp3": "Во-первых на каком основании ты имеешь право ну.. указывать мной и .. я буду делать то что я захочу и .. говориш на меня переводишь на себя поэтому ну посмотри на себя в зеркало.",
    "mne_12_let.mp3": "Вообще-то мне 12 лет. Мне нечем заняться, поэтому я обсираю людей в чате.",
    "vsego_horoshego.mp3": "Всего хорошего."
}

--2025-01-16 23:40:53--  https://storage.yandexcloud.net/homework/hw49.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1747694 (1.7M) [application/zip]
Saving to: ‘hw49.zip’

hw49.zip            100%[===================>]   1.67M   851KB/s    in 2.0s    

2025-01-16 23:40:57 (851 KB/s) - ‘hw49.zip’ saved [1747694/1747694]

Archive:  hw49.zip
  inflating: hw49/bu_ispugalsia.mp3  
  inflating: hw49/mne_12_let.mp3     
  inflating: hw49/pomogi_pj_s_domashkoi.mp3  
  inflating: hw49/strashno_ochen_strashno.mp3  
  inflating: hw49/vsego_horoshego.mp3  
  inflating: hw49/nu_tipa_lol.mp3    
  inflating: hw49/silnaya_nezavisimaya.mp3  


In [ ]:
#@title Вспомогательные функции

# Конвертация MP3 в WAV
def convert_mp3_to_wav(mp3_file, output_dir="converted_audio"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    audio = AudioSegment.from_mp3(mp3_file)
    wav_file = os.path.join(output_dir, os.path.basename(mp3_file).replace('.mp3', '.wav'))
    audio.export(wav_file, format="wav")
    return wav_file


# Добавление шума
def add_noise(audio_file, noise_level=0.02):
    audio = AudioSegment.from_wav(audio_file)
    noise = WhiteNoise().to_audio_segment(duration=len(audio)).apply_gain(-20)  # Генерация белого шума
    noisy_audio = audio.overlay(noise, gain_during_overlay=noise_level * 100)

    # Сохранение в объект BytesIO
    noisy_audio_io = io.BytesIO()
    noisy_audio.export(noisy_audio_io, format='wav')
    noisy_audio_io.seek(0)
    return noisy_audio_io


 # Распознавание речи
def recognize_speech(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio, language="ru-RU")
    except sr.UnknownValueError:
        return "Не удалось распознать речь"
    except sr.RequestError as e:
        return f"Ошибка сервиса: {e}"


 # Оценка качества распознавания
def calculate_accuracy(reference_text, recognized_text):
    ref_words = set(reference_text.lower().split())
    rec_words = set(recognized_text.lower().split())
    if len(ref_words) == 0:
        return 0.0
    common_words = ref_words.intersection(rec_words)
    return len(common_words) / len(ref_words) * 100


# Класс цветов для форматирования текста вывода
class bcolors:
    HEADER = '\033[95m'    # светло-фиолетовый цвет
    OKBLUE = '\033[94m'    # синий цвет
    OKCYAN = '\033[96m'    # голубой цвет
    OKGREEN = '\033[92m'   # зелёный цвет
    WARNING = '\033[93m'   # желтый цвет
    FAIL = '\033[91m'      # красный цвет
    ENDC = '\033[0m'       # сброс всех стилей
    BOLD = '\033[1m'       # жирный шрифт
    UNDERLINE = '\033[4m'  # подчеркнутый текст

# Функция для красочного вывода тестового инференса
def print_colored(text, color, style=""):
    return f"{color}{style}{text}{bcolors.ENDC}"

In [ ]:
#@title  Обработка аудиофайлов

def process_audio_files():
    audio_dir = "/content/hw49"
    mp3_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.mp3')]

    # Словарь эталонных текстов для каждого файла
    reference_texts = REFERENCE

    results = []
    for idx, mp3_path in enumerate(mp3_files, start=1):
        file_name = os.path.basename(mp3_path)
        color_information = print_colored(f"Пример распознавания речи №{idx}:", bcolors.HEADER, bcolors.UNDERLINE)
        print("")
        print(f"{color_information}")
        print("")
        print(f"Обработка файла: {file_name}")
        print("")

        # Воспроизведение аудио
        display(Audio(mp3_path))

        # Конвертация MP3 в WAV
        wav_path = convert_mp3_to_wav(mp3_path)
        print("")

        # Распознавание чистого аудио
        clean_text = recognize_speech(wav_path)
        clean_text_wrapped = textwrap.fill(clean_text, width=60)
        color_clean_text = print_colored(f"Распознанный текст в чистой аудиозаписи:", bcolors.OKBLUE, bcolors.UNDERLINE)
        print(f"{color_clean_text}")
        print(clean_text_wrapped)
        print("")

        # Добавление шума
        noisy_audio = add_noise(wav_path)
        # Распознавание зашумленного аудио
        noisy_text = recognize_speech(noisy_audio)
        noisy_text_wrapped = textwrap.fill(noisy_text, width=60)
        color_noisy_text = print_colored(f"Распознанный текст в зашумленной аудиозаписи:", bcolors.OKBLUE, bcolors.UNDERLINE)
        print(f"{color_noisy_text}")
        print(noisy_text_wrapped)
        print("")

        # Получение эталонного текста
        reference_text = reference_texts.get(file_name, "")
        if not reference_text:
            print(f"Эталонный текст для файла {file_name} не найден. Пропускаем.")
            continue

        # Точность
        clean_accuracy = calculate_accuracy(reference_text, clean_text)
        noisy_accuracy = calculate_accuracy(reference_text, noisy_text)

        results.append({
            "file": file_name,
            "clean_text": clean_text,
            "noisy_text": noisy_text,
            "clean_accuracy": clean_accuracy,
            "noisy_accuracy": noisy_accuracy,
        })

        # Результаты
        color_results = print_colored(f"Результаты распознания речи из файла:", bcolors.FAIL, bcolors.UNDERLINE)
        print(f"{color_results} {file_name}")
        print(f"Точность чистого аудио: {clean_accuracy:.2f}%")
        print(f"Точность зашумленного аудио: {noisy_accuracy:.2f}%")
        print("")
        print(f"="*60)

# Запуск основной функции
if __name__ == "__main__":
    process_audio_files()


Пример распознавания речи №1:

Обработка файла: silnaya_nezavisimaya.mp3




Распознанный текст в чистой аудиозаписи:
во-первых На каком основании ты имеешь право оно указывать
мной и я буду делать то что я захочу и говоришь на меня
переводишь на себя поэтому Ну посмотри на себя в зеркало

Распознанный текст в зашумленной аудиозаписи:
во-первых На каком основании ты имеешь право указывать мной
и я буду делать то что я захочу и говоришь на меня
переводишь на себя поэтому Ну посмотри на себя в зеркало

Результаты распознания речи из файла: silnaya_nezavisimaya.mp3
Точность чистого аудио: 85.19%
Точность зашумленного аудио: 85.19%


Пример распознавания речи №2:

Обработка файла: nu_tipa_lol.mp3




Распознанный текст в чистой аудиозаписи:
и что ну и что типа с этого ну типа Лол как бы Боже типа Лол
ну типа ну как бы типа ну и что типа я имею право типа ну
делать то что Ну типа я захочу как бы ну Лол я считаю что
типа ну если у тебя нормальная самооценка как бы то я ему
Просто понимаешь типа ну как бы ну прикол просто в том что у
меня аргументов нет Боже я просто не собираюсь Ну как бы
типа ну ну ну ну ну мы оправдываться типа

Распознанный текст в зашумленной аудиозаписи:
и что ну и что типа с этого ну типа Лол как бы типа Лол ну
типа ну как бы типа ну и что типа я имею право типа ну
делать то что Ну типа я захочу как бы ну я считаю что типа
ну если у тебя нормальная самооценка как бы то я ему Просто
понимаешь типа ну как ну прикол просто в том что у меня
аргументов Нет я просто не собираюсь Ну как бы типа ну ну ну
ну ну

Результаты распознания речи из файла: nu_tipa_lol.mp3
Точность чистого аудио: 75.61%
Точность зашумленного аудио: 70.73%


Пример распознавания речи №3:

Обраб


Распознанный текст в чистой аудиозаписи:
11 лет мне нечем заняться поэтому я обсираю людей в чате

Распознанный текст в зашумленной аудиозаписи:
11 лет мне нечем заняться поэтому я постираю людей в чате

Результаты распознания речи из файла: mne_12_let.mp3
Точность чистого аудио: 58.33%
Точность зашумленного аудио: 50.00%


Пример распознавания речи №4:

Обработка файла: strashno_ochen_strashno.mp3




Распознанный текст в чистой аудиозаписи:
страшно мы не знаем что это такое если бы мы знали что это
такое мы не знаем что это такое

Распознанный текст в зашумленной аудиозаписи:
страшно мы не знаем что это такое если бы мы знали что это
такое мы не знаем что это такое

Результаты распознания речи из файла: strashno_ochen_strashno.mp3
Точность чистого аудио: 46.15%
Точность зашумленного аудио: 46.15%


Пример распознавания речи №5:

Обработка файла: bu_ispugalsia.mp3




Распознанный текст в чистой аудиозаписи:
Испугался Не бойся я друг я тебя не обижу Иди сюда Иди ко
мне сядь рядом со мной Посмотри мне в глаза ты видишь меня Я
тоже тебя вижу Давай смотреть друг на друга до тех пор пока
наши глаза не устанут ты не хочешь Почему что-то не так

Распознанный текст в зашумленной аудиозаписи:
Испугался Не бойся я друг я тебя не обижу Иди сюда Иди ко
мне сядь рядом со мной Посмотри мне в глаза ты видишь меня Я
тоже тебя вижу Давай смотреть друг на друга до тех пор пока
наши глаза не устанут ты не хочешь Почему что-то не так

Результаты распознания речи из файла: bu_ispugalsia.mp3
Точность чистого аудио: 60.98%
Точность зашумленного аудио: 60.98%


Пример распознавания речи №6:

Обработка файла: vsego_horoshego.mp3




Распознанный текст в чистой аудиозаписи:
хорошего

Распознанный текст в зашумленной аудиозаписи:
хорошего

Результаты распознания речи из файла: vsego_horoshego.mp3
Точность чистого аудио: 0.00%
Точность зашумленного аудио: 0.00%


Пример распознавания речи №7:

Обработка файла: pomogi_pj_s_domashkoi.mp3




Распознанный текст в чистой аудиозаписи:
помоги пожалуйста с домашкой помоги пожалуйста Я обвела то
что надо пожалуйста помоги помоги Напиши ответ помоги
пожалуйста

Распознанный текст в зашумленной аудиозаписи:
пожалуйста

Результаты распознания речи из файла: pomogi_pj_s_domashkoi.mp3
Точность чистого аудио: 41.18%
Точность зашумленного аудио: 5.88%



В данном задании для оценки качества распознавания речи на основе чистого и зашумленного аудио была выбрана следующая метрика:

 * **Точность распознавания речи (Accuracy):**

Точность была выбрана как метрика для оценки того, насколько точно распознается текст, произнесенный в аудио, относительно эталонного текста.

Точность определяется для двух типов аудио:
   1. "Чистое аудио" без добавления шума
   2. "Зашумленное аудио" с добавлением белого шума

* **Почему выбрана именно эта метрика:**

Точность является наглядным способом для количественной оценки качества распознавания текста. Основное внимание в данном случае уделяется процентному совпадению слов из распознанного текста с эталонным. Это позволяет четко понять, насколько сильно добавление шума влияет на распознавание речи.

**Результаты метрик для каждой записи:**

1. silnaya_nezavisimaya.mp3:
  * Точность чистого аудио: 85.19%
  * Точность зашумленного аудио: 85.19%

Вывод: Добавление шума не повлияло на точность распознавания. Это может свидетельствовать о том, что аудиофайл с чистым звуком и зашумленный файл имеют схожие особенности речи, или шум был незначительным.

2. nu_tipa_lol.mp3:
  * Точность чистого аудио: 75.61%
  * Точность зашумленного аудио: 70.73%

Вывод: Снижение точности при добавлении шума на ~5% указывает на то, что шум немного затруднил распознавание речи, особенно в сложных речевых оборотах, с большим количеством “типичных” фраз и бессмысленных слов.

3. mne_12_let.mp3:
  * Точность чистого аудио: 58.33%
  * Точность зашумленного аудио: 50.00%

Вывод: Точность распознавания снизилась на ~8% при добавлении шума. Это также свидетельствует о влиянии шума на способность системы правильно распознавать слова и цифры, особенно когда речь менее четкая и фраза сложна.

4. strashno_ochen_strashno.mp3:
  * Точность чистого аудио: 46.15%
  * Точность зашумленного аудио: 46.15%

Вывод: В данном случае точность осталась на том же уровне. Это может означать, что данный аудиофайл и так был достаточно “шумным” или сложным для распознавания, и добавление дополнительного шума не оказало значительного влияния.

5. bu_ispugalsia.mp3:
  * Точность чистого аудио: 60.98%
  * Точность зашумленного аудио: 60.98%

Вывод: Никаких изменений в точности не произошло. Вероятно, шум был слишком незначителен, чтобы повлиять на распознавание речи, особенно в контексте простых и четких фраз.

6. vsego_horoshego.mp3:
  * Точность чистого аудио: 0.00%
  * Точность зашумленного аудио: 0.00%

Вывод: Здесь и чистое, и зашумленное аудио не были распознаны. Это может быть связано с коротким фрагментом или с произношением фразы по слогам, что затрудняет распознавание речи.

7. pomogi_pj_s_domashkoi.mp3:
  * Точность чистого аудио: 41.18%
  * Точность зашумленного аудио: 5.88%

Вывод: Значительное снижение точности при добавлении шума указывает на то, что шум сильно повлиял на распознавание. В данном случае, речь была исполнена шепотом, и добавление шума сделало распознавание невозможным.


**Общие выводы:**

 * В большинстве случаев добавление шума снижает точность распознавания, особенно в тех случаях, где изначальная точность была не высокой.

 * Наибольшее снижение точности наблюдается в случае с фрагментами, где речь менее четкая или слишком быстрая, как в “pomogi_pj_s_domashkoi.mp3”.

 * В некоторых случаях, например в “silnaya_nezavisimaya.mp3”, добавление шума не оказывает существенного влияния, что может быть связано с хорошей разборчивостью речи на исходном аудио.

 * Для некоторых файлов, например “vsego_horoshego.mp3”, добавление шума не изменяет результат, поскольку исходное аудио было практически не распознаваемым.

Таким образом, шум оказывает заметное влияние на точность распознавания, особенно когда речь сложная или плохо разборчивая.